# BoXHED 2.0 quick start

[BoXHED 2.0](https://arxiv.org/pdf/2103.12591.pdf) is a software package for estimating hazard functions nonparametrically via gradient boosting. It dramatically improves upon BoXHED 1.0 [BoXHED: Boosted eXact Hazard Estimator with Dynamic covariates](http://proceedings.mlr.press/v119/wang20o/wang20o.pdf) in speed. BoXHED 2.0 also allows for more general forms of survival data including recurrent events.

This tutorial demonstrates how to apply BoXHED 2.0 to a synthetic dataset. An expanded user's guide can be found in main.py.

### 1. Importing convenience functions from main.py

**_read_synth** reads the synthetic training data and returns a pandas dataframe.

Input:
* *exp_num*: index for the simulated dataset used in Section 5.3 of the BoXHED 1.0 paper (1, 2, 3, or 4)
* *num_irr*: number of irrelevant covariates used in the simulation (0, 20, or 40)

Output:

A pandas data frame with the following columns:
* *subject*: subject ID
* *t_start*: the start time of an epoch for the patient
* *t_end*: the end time of the epoch
* *X_i*: Values of covariates between *t_start* and *t_end* 

We will see what the input data looks like below

In [1]:
from main import _read_synth

**_read_synth_test** reads the synthetic test data and returns a pandas dataframe. The values of the true hazard function are also provided for accuracy comparisons

Input:
* *exp_num*: index for the simulated dataset used in Section 5.3 of the BoXHED 1.0 paper (1, 2, 3, or 4)
* *num_irr*: number of irrelevant covariates used in the simulation (0, 20, or 40)

Output:

* a pandas data frame with the following columns:
  * *t*: time
  * *X_i*: covariate values at time *t*
* a numpy array containing the values of the true hazard function for each row above


In [2]:
from main import _read_synth_test

### 2. Running an example

In [3]:
exp_num       = 1  
num_irr       = 20  
num_quantiles = 256 #number of candidate split points (locations based on quantiles). Integer from 8 to 256

data = _read_synth(exp_num, num_irr)

In [4]:
data

,subject,t_start,t_end,X_0,X_1,X_2,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,delta
0,1.0,0.010000,0.064333,0.152407,0.766206,0.243241,0.964592,0.138555,0.328829,0.677724,0.244175,0.330192,0.748040,0.622986,0.177698,0.104307,0.243527,0.0
1,1.0,0.064333,0.135136,0.308475,0.062914,0.099524,0.368705,0.801284,0.428748,0.136333,0.260494,0.694811,0.801564,0.445232,0.729579,0.442358,0.930640,0.0
2,1.0,0.194810,0.223106,0.614977,0.506616,0.136171,0.702439,0.914293,0.376288,0.132540,0.249119,0.478678,0.902935,0.536759,0.484972,0.441427,0.099009,1.0
3,1.0,0.223106,0.248753,0.614977,0.506616,0.136171,0.702439,0.914293,0.376288,0.132540,0.249119,0.478678,0.902935,0.536759,0.484972,0.441427,0.099009,0.0
4,1.0,0.248753,0.290695,0.879444,0.991845,0.821709,0.598007,0.562564,0.808424,0.753449,0.205437,0.897105,0.466843,0.518021,0.863728,0.492777,0.921254,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76259,5000.0,0.795027,0.841729,0.196407,0.025375,0.387692,0.641372,0.082344,0.852261,0.214533,0.354664,0.220370,0.220250,0.132187,0.839544,0.855417,0.202244,1.0
76260,5000.0,0.841729,0.886587,0.196407,0.025375,0.387692,0.641372,0.082344,0.852261,0.214533,0.354664,0.220370,0.220250,0.132187,0.839544,0.855417,0.202244,0.0
76261,5000.0,0.886587,0.949803,0.671227,0.986347,0.972556,0.350481,0.776933,0.851917,0.763872,0.604797,0.805040,0.838278,0.170575,0.166778,0.682828,0.411376,0.0
76262,5000.0,0.991068,0.997355,0.506519,0.931181,0.018766,0.716787,0.958361,0.158138,0.343687,0.437030,0.220983,0.871205,0.852959,0.130933,0.532446,0.690477,0.0


As can be seen above, $t_{start}<t_{end}$ for each epoch (row). Also, the beginning of one epoch starts no earlier than the end of the previous one, i.e. $t_{{end}_{i}}\leq t_{{start}_{i+1}}$. Delta denotes whether an event (possibly recurrent) occurred at the end of the epoch.

For covariates with missing values, BoXHED 2.0 implements tree splits of the form

Left daughter node: {x<=split.point or x is missing}; Right daughter node: {x>split.point}

or

Left daughter node: {x<=split.point}; Right daughter node: {x>split.point or x is missing}.

Alternatively, the user may choose to impute the missing values, for example by carrying forward the most recent observed value.

Creating an instance of BoXHED to preprocess the training data. boxhed.preprocess() takes 4 arguments:

* *num_quantiles*: the number of candidate split points to try for time and for each covariate. The locations of the split points are based on the quantiles of the training data
* *is_cat*: a list of the column indexes that contain categorical data. The categorical data must be one-hot encoded. For example, if a categorical variable with 3 factors is one hot encoded into binary-valued columns 4,5,6, then is_cat = [4,5,6]
* *weighted*: if set to True, the locations of the candidate split points will be based on weighted quantiles (see Section 3.3 of the BoXHED 2.0 paper)
* *nthreads*: number of CPU threads to use for preprocessing the data

Output:

* *subject*: subject ID for each row in the data frames X, w, and delta
* *X*: each row represents an epoch of the transformed data, and contains the values of the covariates as well as its start time
* *w*: Length of each epoch
* *delta*: Equals one if an event occurred at the end of the epoch; zero otherwise


In [5]:
from boxhed import boxhed
boxhed_ = boxhed(max_depth    = 1,
                 n_estimators = 150)

subjects, X, w, delta = boxhed_.preprocess(
        data             = data,
        num_quantiles    = num_quantiles,
        is_cat           = [],
        weighted         = False,
        nthreads         = 1)

Perform *K*-fold cross-validation to choose the hyperparameters {number of boosted trees, tree depth, learning rate}.

First, specify the candidate values for the hyperparameters to cross-validate on (more trees and/or deeper trees may be needed for other datasets):

In [6]:
param_grid = {'max_depth':    [1, 2, 3, 4, 5],
              'n_estimators': [50, 100, 150, 200, 250, 300],
              'eta':          [0.1]}

Next, specify:

* The list of GPU IDs to use for training (we set gpu_list to [-1] to use CPU in this tutorial)
* Training batch size, which is the maximum number of BoXHED2.0 instances trained at any point in time. If we perform 10-fold cross-validation using the above param_grid, we would need to train $5\times 6 \times 10 =300$ instances in total
    * When using GPUs, each GPU will train at most batch_size/len(gpu_list) instances at a time
    * When gpu_list = [-1], batch_size is the number of CPU threads used, with each one training one instance at a time
* The value of *K* in *K*-fold cross-validation

In [7]:
gpu_list = [-1]
batch_size = 6
num_folds = 5

Call the *cv* function to perform *K*-fold cross validation on the training set. This outputs the cross validation results for the different hyperparameter combinations
* *cv_rslts*: mean and st.dev of the log-likelihood value for each hyperparameter combination
* *best_params*: The hyper-parameter combination where the mean log-likelihood value is maximized. *HOWEVER*, we recommend using the one-standard-error rule to select the most parsimonious combination that is within st.dev/$\sqrt{k}$ of the maximum log-likelihood value ($\S$7.10 of Hastie et al. (2009))

Note: Due to a problem the Jupyter notebook has with the multiprocessing package, the code commented out below should be run as a script outside of Jupyter, for example in Spyder. Here we set the hyperparameters to the combination found using 5-fold cross-validation on the data

In [8]:
'''
from model_selection import cv
cv_rslts, best_params = cv(
                          param_grid, 
                          X, 
                          w,
                          delta,
                          subjects, 
                          num_folds,
                          gpu_list,
                          batch_size)
'''
best_params = {'max_depth':    1,
              'n_estimators': 150,
              'eta':          0.1}

Fit BoXHED to the training data:

In [9]:
boxhed_.set_params (**best_params)
boxhed_.fit (X, delta, w)

boxhed(n_estimators=150)

Load the test set and the values of the true hazard function at the test points:

In [10]:
true_haz, test_x = _read_synth_test(exp_num, num_irr) 

Use the fitted model to estimate the value of the hazard function for each row of the test set:

In [11]:
preds = boxhed_.predict(test_x)

Compute the RMSE of the estimates, and its 95% confidence interval:

In [12]:
from main import calc_L2
L2 = calc_L2(preds, true_haz)
L2

[0.19378179771296392, [0.19282653084500834, 0.1947370645809195]]